<div>
<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=220 height=220 align=left class="saturate">

<br>
<font face="Times New Roman">
<div dir=ltr align=center> 
<!-- <font color=0F5298 size=7> -->
<font color=0F5298 size=6>
    Introduction to Machine Learning <br> <br>
<!-- <font color=2565AE size=5> -->
<font size=5>
    Computer Engineering Department <br>
    Spring 2023 <br> <br>
<font color=606060 size=5>
    Homework 2: Practical - LDA & QDA <br> <br>
<font color=686880 size=4>
    TAs: Arman Malekzadeh - Arshia Soltani Moakhar
    
____

### Full Name : Reza Soumi
### Student Number : 98105857
___

In [69]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

# Exploration (20 points)

In [70]:
!wget https://github.com/sharif-ml/sharif-ml.github.io/raw/main/assignments/dataset-train.csv

--2023-04-13 13:02:16--  https://github.com/sharif-ml/sharif-ml.github.io/raw/main/assignments/dataset-train.csv
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sharif-ml/sharif-ml.github.io/main/assignments/dataset-train.csv [following]
--2023-04-13 13:02:17--  https://raw.githubusercontent.com/sharif-ml/sharif-ml.github.io/main/assignments/dataset-train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89401 (87K) [text/plain]
Saving to: ‘dataset-train.csv.1’

dataset-train.csv.1 100%[===================>]  87.31K  --.-KB/s    in 0.02s   

2023-04-13 13:02:17 (5.02 MB/s) - ‘dataset-train.c

Load the dataset as a dataframe

In [73]:
df = pd.read_csv('dataset-train.csv')

print(df.head())

   Daily Time Spent on Site  Age  Area Income  Daily Internet Usage  \
0                     68.95   35     61833.90                256.09   
1                     80.23   31     68441.85                193.77   
2                     69.47   26     59785.94                236.50   
3                     74.15   29     54806.18                245.89   
4                     68.37   35     73889.99                225.58   

                           Ad Topic Line            City     Country  \
0     Cloned 5thgeneration orchestration     Wrightburgh     Tunisia   
1     Monitored national standardization       West Jodi       Nauru   
2       Organic bottom-line service-desk        Davidton  San Marino   
3  Triple-buffered reciprocal time-frame  West Terrifurt       Italy   
4          Robust logistical utilization    South Manuel     Iceland   

             Timestamp  Clicked on Ad  Gender  
0  2016-03-27 00:53:11              0  female  
1  2016-04-04 01:39:02              0    mal

Get the minimum "Area Income" for all Italians

In [74]:
min_area_income = df.loc[df['Country'] == 'Italy', 'Area Income'].min()
min_area_income

54286.1

Make a new dataframe called "sample_df" by sampling 100 rows from the previously made dataframe


In [75]:
sample_df = df.sample(n=100, random_state=42)
sample_df

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Country,Timestamp,Clicked on Ad,Gender
696,39.19,54,52581.16,173.05,Stand-alone empowering benchmark,Leahside,Guatemala,2016-01-23 21:15:57,1,female
667,79.61,31,58342.63,235.97,Customizable value-added project,Luisfurt,Monaco,2016-04-04 21:23:13,0,female
63,86.06,32,61601.05,178.92,Focused upward-trending core,North Leonmouth,Bulgaria,2016-03-31 20:55:22,0,male
533,91.43,39,46964.11,209.91,Organic motivating model,West Carmenfurt,French Polynesia,2016-07-07 03:55:01,0,male
66,63.89,40,51317.33,105.22,Synchronized user-facing core,Jensenborough,Rwanda,2016-01-08 00:17:27,1,female
...,...,...,...,...,...,...,...,...,...,...
608,62.06,40,63493.60,116.27,Digitized radical array,East Rachaelfurt,Jersey,2016-03-16 20:10:53,1,male
705,78.84,26,56593.80,217.66,Persistent even-keeled application,Shawnside,Italy,2016-06-29 07:20:46,0,male
754,76.87,36,72948.76,212.59,Down-sized bandwidth-monitored core,Mullenside,South Georgia and the South Sandwich Islands,2016-07-08 22:30:10,0,female
54,62.31,47,62722.57,119.30,Team-oriented encompassing portal,Amandahaven,Turkey,2016-02-14 07:36:58,1,female


Compare all men and women aged less than 30 years in terms of their average "Daily Internet Usage". Your result should contain the average daily usage for each age and gender, seperately.

In [76]:
filtered_data = df[(df['Age'] < 30) & (df['Gender'].isin(['male', 'female']))]

avg_usage = filtered_data.groupby(['Gender', 'Age'])['Daily Internet Usage'].mean()

print(avg_usage)

Gender  Age
female  19     184.940000
        20     202.160000
        21     186.980000
        22     200.070000
        23     187.105714
        24     178.693333
        25     193.618571
        26     192.741875
        27     189.157692
        28     205.873333
        29     191.191200
male    19     189.523333
        20     198.982500
        21     212.815000
        22     188.837500
        23     184.615000
        24     200.055556
        25     206.527857
        26     203.916250
        27     205.916154
        28     196.416842
        29     202.005714
Name: Daily Internet Usage, dtype: float64


# Preprocessing (30 points)

Remove the following columns: "Ad Topic Line", "Timestamp"

In [77]:
data = df.drop(['Ad Topic Line', 'Timestamp'], axis=1)

print(data)

     Daily Time Spent on Site  Age  Area Income  Daily Internet Usage  \
0                       68.95   35     61833.90                256.09   
1                       80.23   31     68441.85                193.77   
2                       69.47   26     59785.94                236.50   
3                       74.15   29     54806.18                245.89   
4                       68.37   35     73889.99                225.58   
..                        ...  ...          ...                   ...   
795                     75.64   29     69718.19                204.82   
796                     79.22   27     63429.18                198.79   
797                     77.05   34     65756.36                236.08   
798                     66.83   46     77871.75                196.17   
799                     76.20   24     47258.59                228.81   

                 City     Country  Clicked on Ad  Gender  
0         Wrightburgh     Tunisia              0  female  
1    

Convert the "City" and "Country" columns to their one-hot encoded versions.
For each set of one-hot encoded columns, remove one of them to make the columns not correlated. Finally, remove the original columns named "City" and "Country".

In [78]:
df = pd.get_dummies(data, columns=["City", "Country"])

In [79]:
df = df.drop(columns=["City_Adamsbury", "Country_Uzbekistan"])
df

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Clicked on Ad,Gender,City_Alanview,City_Alexanderfurt,City_Alexanderview,City_Alexisland,...,Country_United States of America,Country_Uruguay,Country_Vanuatu,Country_Venezuela,Country_Vietnam,Country_Wallis and Futuna,Country_Western Sahara,Country_Yemen,Country_Zambia,Country_Zimbabwe
0,68.95,35,61833.90,256.09,0,female,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,80.23,31,68441.85,193.77,0,male,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,69.47,26,59785.94,236.50,0,female,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,74.15,29,54806.18,245.89,0,male,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,68.37,35,73889.99,225.58,0,female,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,75.64,29,69718.19,204.82,0,male,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
796,79.22,27,63429.18,198.79,0,male,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
797,77.05,34,65756.36,236.08,0,female,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
798,66.83,46,77871.75,196.17,0,male,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Convert "Gender" column to a new binary column (having only 0 and 1 values). Then remove the original column.

In [80]:
df['Gender'] = df['Gender'].map({'male': 1, 'female': 0})

In [81]:
df

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Clicked on Ad,Gender,City_Alanview,City_Alexanderfurt,City_Alexanderview,City_Alexisland,...,Country_United States of America,Country_Uruguay,Country_Vanuatu,Country_Venezuela,Country_Vietnam,Country_Wallis and Futuna,Country_Western Sahara,Country_Yemen,Country_Zambia,Country_Zimbabwe
0,68.95,35,61833.90,256.09,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,80.23,31,68441.85,193.77,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,69.47,26,59785.94,236.50,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,74.15,29,54806.18,245.89,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,68.37,35,73889.99,225.58,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,75.64,29,69718.19,204.82,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
796,79.22,27,63429.18,198.79,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
797,77.05,34,65756.36,236.08,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
798,66.83,46,77871.75,196.17,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [82]:
df = df.drop(columns=["Gender"])
df

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Clicked on Ad,City_Alanview,City_Alexanderfurt,City_Alexanderview,City_Alexisland,City_Aliciatown,...,Country_United States of America,Country_Uruguay,Country_Vanuatu,Country_Venezuela,Country_Vietnam,Country_Wallis and Futuna,Country_Western Sahara,Country_Yemen,Country_Zambia,Country_Zimbabwe
0,68.95,35,61833.90,256.09,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,80.23,31,68441.85,193.77,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,69.47,26,59785.94,236.50,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,74.15,29,54806.18,245.89,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,68.37,35,73889.99,225.58,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,75.64,29,69718.19,204.82,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
796,79.22,27,63429.18,198.79,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
797,77.05,34,65756.36,236.08,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
798,66.83,46,77871.75,196.17,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


By dropping the "Clicked on Ad" column, make a new dataframe called "features_df"

In [83]:
features_df = df.drop(columns=["Clicked on Ad"])
features_df

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,City_Alanview,City_Alexanderfurt,City_Alexanderview,City_Alexisland,City_Aliciatown,City_Alvaradoport,...,Country_United States of America,Country_Uruguay,Country_Vanuatu,Country_Venezuela,Country_Vietnam,Country_Wallis and Futuna,Country_Western Sahara,Country_Yemen,Country_Zambia,Country_Zimbabwe
0,68.95,35,61833.90,256.09,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,80.23,31,68441.85,193.77,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,69.47,26,59785.94,236.50,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,74.15,29,54806.18,245.89,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,68.37,35,73889.99,225.58,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,75.64,29,69718.19,204.82,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
796,79.22,27,63429.18,198.79,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
797,77.05,34,65756.36,236.08,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
798,66.83,46,77871.75,196.17,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
print(len(list(features_df.columns.values)))

1015


Make a new dataframe called "target_df" containing only the "Clicked on Ad" column

In [85]:
target_df = df[['Clicked on Ad']]
target_df

,Clicked on Ad
0,0
1,0
2,0
3,0
4,0
...,...
795,0
796,0
797,0
798,0


Make two numpy arrays called "x" and "y" from "features_df" and "target_df" dataframes, respectively.

In [86]:
x = features_df.to_numpy()
y = np.ravel(target_df.to_numpy())

# Training Models (40 points)

Split the dataset (`x` and `y`) into "training" and "testing" parts containing 80% and 20% of the data respectively.

In [87]:
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2)

Train both LDA and QDA models on the data using `sklearn`

In [93]:
lda = LinearDiscriminantAnalysis()
lda.fit(x_train, y_train)

LinearDiscriminantAnalysis()

In [92]:
qda = QuadraticDiscriminantAnalysis()
qda.fit(x_train, y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


QuadraticDiscriminantAnalysis()

Train LDA and QDA using your own code!

Linear Discriminant Analysis

In [94]:
import numpy as np

class LinearDiscriminantAnalysis:
    """
    Fits it on data, then uses predict to get results.
    """

    def __init__(self):
        pass
    
    def fit(self, X, y):
        """Fit the training data with finding Sw and Sb
        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
            Features
        y : array-like, shape = [n_samples, n_labels]
            Labels
        Returns
        -------
        self : object
        """

        n_samples, n_features = X.shape
        
        self.class_means = []
        for label in np.unique(y):
            self.class_means.append(np.mean(X[y == label], axis=0))
        self.class_means = np.array(self.class_means)
        
        self.Sw = np.zeros((n_features, n_features))
        for label in np.unique(y):
            class_scatter = np.zeros((n_features, n_features))
            for sample in X[y == label]:
                sample, mean = sample.reshape(n_features, 1), self.class_means[label].reshape(n_features, 1)
                class_scatter += (sample - mean).dot((sample - mean).T)
            self.Sw += class_scatter
        
        overall_mean = np.mean(X, axis=0)
        self.overall_mean = overall_mean
        self.Sb = np.zeros((n_features, n_features))
        for i, mean_vec in enumerate(self.class_means):
            n = X[y==i,:].shape[0]
            mean_vec = mean_vec.reshape(n_features, 1) # make column vector
            overall_mean = overall_mean.reshape(n_features, 1) # make column vector
            self.Sb += n * (mean_vec - overall_mean).dot((mean_vec - overall_mean).T)
        
        if np.linalg.det(self.Sw) == 0:
          self.Sw += 0.0001 * np.identity(self.Sw.shape[0])

        eig_vals, eig_vecs = np.linalg.eig(np.linalg.inv(self.Sw).dot(self.Sb))
        
        self.w = eig_vecs[:, np.argsort(eig_vals)[::-1][:len(np.unique(y)) - 1]]
        self.centroid = np.dot(self.class_means, self.w)

        return self
    
    def predict(self, X):
        """ Predicts the value after the model has been trained.
        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
            Features
        Returns
        -------
        Predicted value
        """
        projection = np.dot(X, self.w)
        result = []
        for projected in projection:
            result.append(np.argmin([np.linalg.norm(projected-centroid) for centroid in self.centroid]))
        return result

Quadratic Discriminant Analysis (Optional) (15 bonus points)

In [96]:
class QuadraticDiscriminantAnalysis:
    """
    Fits it on data, then uses predict to get results.
    """

    def __init__(self):
        pass
    
    def fit(self, X, t):
        self.priors = dict()
        self.means = dict()
        self.covs = dict()
        
        self.classes = np.unique(t)
        
        def take_cov(row,mean):
            return (row.reshape(-1,1) - mean).dot((row.reshape(-1,1) - mean).T)
   
        for c in self.classes:
            X_c = X[t == c]
            self.priors[c] = X_c.shape[0] / X.shape[0]
            self.means[c] = np.mean(X_c, axis=0)
            self.covs[c] = np.cov(X_c, rowvar=False)
            if np.linalg.det(self.covs[c]) == 0:
                  self.covs[c] += 0.001 * np.identity(self.covs[c].shape[0])
            # self.covs[c] = (1./(X_c.shape[0] - len(self.classes))) * (sum([take_cov(row,self.means[c]) for row in X_c]))
      
    def predict(self, X):
        preds = list()
        for x in X:
            posts = list()
            for c in self.classes:
                prior = np.log(self.priors[c])
                inv_cov = np.linalg.inv(self.covs[c])
                inv_cov_det = np.linalg.det(inv_cov)
                #print(inv_cov_det)
                diff = x-self.means[c]
                likelihood = -0.5*diff.T @ inv_cov @ diff
                # + 0.5*np.log(inv_cov_det)
                post = prior + likelihood
                posts.append(post)
            pred = self.classes[np.argmax(posts)]
            preds.append(pred)
        return np.array(preds)

In [97]:
# Train your own models
my_lda = LinearDiscriminantAnalysis()
my_lda.fit(x_train, y_train)

my_qda = QuadraticDiscriminantAnalysis()
my_qda.fit(x_train, y_train)

# Evaluation (10 points)

Using all 4 models, predict the labels for all samples belonging to the "testing" part

In [99]:
y_pred_sklda = lda.predict(x_test)
y_pred_skqda = qda.predict(x_test)
y_pred_lda = my_lda.predict(x_test)
y_pred_qda = my_qda.predict(x_test)

/usr/local/lib/python3.9/dist-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)


Calculate Accuracy, Recall, and F1-Scores

In [103]:
from sklearn.metrics import accuracy_score, recall_score, f1_score

print("skleanr evaluation: ")
# Calculate accuracy score
accuracy_sklda = accuracy_score(y_test, y_pred_sklda)
accuracy_skqda = accuracy_score(y_test, y_pred_skqda)
print("Accuracy sklearn LDA:", accuracy_sklda)
print("Accuracy sklearn QDA:", accuracy_skqda)

# Calculate recall score
recall_sklda = recall_score(y_test, y_pred_sklda)
recall_skqda = recall_score(y_test, y_pred_skqda)
print("Recall sklearn LDA:", recall_sklda)
print("Recall sklearn QDA:", recall_skqda)

# Calculate F1 score
f1_sklda = f1_score(y_test, y_pred_sklda)
f1_skqda = f1_score(y_test, y_pred_skqda)
print("F1-score sklearn LDA:", f1_sklda)
print("F1-score sklearn QDA:", f1_skqda)


print("\nmy model evaluation: ")
# Calculate accuracy score
accuracy_lda = accuracy_score(y_test, y_pred_lda)
accuracy_qda = accuracy_score(y_test, y_pred_qda)
print("Accuracy LDA:", accuracy_lda)
print("Accuracy QDA:", accuracy_qda)

# Calculate recall score
recall_lda = recall_score(y_test, y_pred_lda)
recall_qda = recall_score(y_test, y_pred_qda)
print("Recall LDA:", recall_lda)
print("Recall QDA:", recall_qda)

# Calculate F1 score
f1_lda = f1_score(y_test, y_pred_lda)
f1_qda = f1_score(y_test, y_pred_qda)
print("F1-score LDA:", f1_lda)
print("F1-score QDA:", f1_qda)


skleanr evaluation: 
Accuracy sklearn LDA: 0.9
Accuracy sklearn QDA: 0.775
Recall sklearn LDA: 0.8395061728395061
Recall sklearn QDA: 0.9876543209876543
F1-score sklearn LDA: 0.8947368421052632
F1-score sklearn QDA: 0.8163265306122449

my model evaluation: 
Accuracy LDA: 0.9375
Accuracy QDA: 0.7
Recall LDA: 0.9012345679012346
Recall QDA: 0.7901234567901234
F1-score LDA: 0.935897435897436
F1-score QDA: 0.7272727272727273


Compute the ROC AUC Score using `sklearn` (for both LDA and QDA models trained using this library)

In [104]:
from sklearn.metrics import roc_auc_score

print("sklearn evaluation:")
# calculate the ROC AUC score
roc_auc_sklda = roc_auc_score(y_test, y_pred_sklda)
roc_auc_skqda = roc_auc_score(y_test, y_pred_skqda)

# print the ROC AUC score
print(f"ROC AUC score sklearn LDA: {roc_auc_sklda:.3f}")
print(f"ROC AUC score sklearn QDA: {roc_auc_skqda:.3f}")

print("\nmy model evaluation:")
# calculate the ROC AUC score
roc_auc_lda = roc_auc_score(y_test, y_pred_lda)
roc_auc_qda = roc_auc_score(y_test, y_pred_qda)

# print the ROC AUC score
print(f"ROC AUC score sklearn LDA: {roc_auc_lda:.3f}")
print(f"ROC AUC score sklearn QDA: {roc_auc_qda:.3f}")

sklearn evaluation:
ROC AUC score sklearn LDA: 0.901
ROC AUC score sklearn QDA: 0.772

my model evaluation:
ROC AUC score sklearn LDA: 0.938
ROC AUC score sklearn QDA: 0.699


# Kaggle Competition

In [126]:
df = pd.read_csv('assignment2-test-data.csv')
data = df.drop(['Ad Topic Line', 'Timestamp', 'ID'], axis=1)
df = pd.get_dummies(data, columns=["City", "Country"])
# add missing columns in test data
missing_cols = set(features_df.columns.values) - set(df.columns.values)
print(len(missing_cols))
print(len(df.columns))
print(df.columns)
print(missing_cols)
#for c in missing_cols:
#    df[c] = 0
print(len(list(features_df.columns.values)))
df['Gender'] = df['Gender'].map({'male': 1, 'female': 0})
df = df.drop(columns=["Gender"])
x_test_kaggle = df.to_numpy()
#y_pred_lda_kaggle = my_lda.predict(x_test_kaggle)
#print(y_pred_lda_kaggle)

865
345
Index(['Daily Time Spent on Site', 'Age', 'Area Income',
       'Daily Internet Usage', 'Gender', 'City_Adamside', 'City_Adamsstad',
       'City_Alexandrafort', 'City_Alvarezland', 'City_Angelhaven',
       ...
       'Country_United States of America', 'Country_Uruguay',
       'Country_Uzbekistan', 'Country_Vanuatu', 'Country_Venezuela',
       'Country_Vietnam', 'Country_Wallis and Futuna',
       'Country_Western Sahara', 'Country_Yemen', 'Country_Zimbabwe'],
      dtype='object', length=345)
{'City_Lake Cynthia', 'City_West Leahton', 'City_Lake Joshuafurt', 'Country_Germany', 'City_Johnsonview', 'City_Robertsonburgh', 'Country_Pitcairn Islands', 'City_Destinyfurt', 'City_Pottermouth', 'City_Parkerhaven', 'City_West Justin', 'Country_India', 'City_New Williamville', 'City_Brownport', 'City_Lake Ian', 'City_West Angela', 'City_New Jeffreychester', 'City_Smithburgh', 'City_Hayesmouth', 'City_Lake Melindamouth', 'Country_Bouvet Island (Bouvetoya)', 'City_New Sharon', 'City_Po

Link: https://www.kaggle.com/t/988b0aa2db2446fd85cff571d69c6386